In [86]:
import sys
import os
import numpy as np
import dadi
from dadi import Numerics, PhiManip, Integration, Misc
from dadi.Spectrum_mod import Spectrum
import pylab
import matplotlib.pyplot as plt
import argparse
import pandas

## Build a dictionary of the best model and its parameters

Choose the population pair to investigate:

In [58]:
pops = "sm_ki"

Import all of the optimization results:

In [59]:
opti_result_dir = "/Users/enrico/Documents/Lynx-Introgression-Scans/2-infer_demographic_history/dadi/tables/"

all_best_lls = []

for chosen_model in ["model_1_a", "model_1_b", "model_2_a", "model_2_b", "model_2_c"]:
    
    for i in range(1, 21):
    
        model_file_name = [opti_result_dir, pops, ".", chosen_model, ".optimized.r_", str(i), ".txt"]
        model_file_name = ''.join(model_file_name)
        if os.path.isfile(model_file_name):
            model_optimized_allrounds = pandas.read_csv(model_file_name, sep='\t')
            ll_col = model_optimized_allrounds.loc[:,"log-likelihood"]
            best_ll = model_optimized_allrounds.iloc[ll_col.idxmax(),]
            all_best_lls.append(best_ll)

**RUN THIS ONLY FOR TESTING WITH RESULTS FROM model_2_d :**

In [210]:
opti_result_dir = "/Users/enrico/Documents/Lynx-Introgression-Scans/2-infer_demographic_history/dadi/tables/"

all_best_lls = []

for chosen_model in ["model_2_d"]:
    
    for i in range(1, 21):
    
        model_file_name = [opti_result_dir, pops, ".", chosen_model, ".optimized.r_", str(i), ".txt"]
        model_file_name = ''.join(model_file_name)
        if os.path.isfile(model_file_name):
            model_optimized_allrounds = pandas.read_csv(model_file_name, sep='\t')
            ll_col = model_optimized_allrounds.loc[:,"log-likelihood"]
            best_ll = model_optimized_allrounds.iloc[ll_col.idxmax(),]
            all_best_lls.append(best_ll)

Extract best scoring model and its parameters:

In [211]:
all_best_lls_df = pandas.concat(all_best_lls, axis=1).T
best_ll = all_best_lls_df.sort_values('log-likelihood', ascending=False)[:1]
if best_ll.iloc[0,0] == "model_1_a":
    param_string = best_ll.iloc[0,6].split(",")
if best_ll.iloc[0,0] == "model_1_b":
    param_string = best_ll.iloc[0,6].split(",")
if best_ll.iloc[0,0] == "model_2_a":
    param_string = best_ll.iloc[0,6].split(",")
if best_ll.iloc[0,0] == "model_2_b":
    param_string = best_ll.iloc[0,6].split(",")
if best_ll.iloc[0,0] == "model_2_c":
    param_string = best_ll.iloc[0,6].split(",")
if best_ll.iloc[0,0] == "model_2_d":
    param_string = best_ll.iloc[0,6].split(",")
params = [float(l) for l in param_string]

Build parameter dictionary:

In [212]:
if len(params) == 9:
    param_def = ["Tsplit", "Tbot", "iber_a", "iber_pr", "eura_a", "eura_pr"]
if len(params) == 13:
    param_def = ["Tsplit", "Tbot", "Tbot_a", "iber_a", "iber_pr_a", "iber_pr", "eura_a", "eura_pr"]
par_dict = {param_def[i]: params[i] for i in range(len(param_def))}
par_dict['theta'] = best_ll.iloc[0,5]
par_dict['best_model'] = best_ll.iloc[0,0]
par_dict

{'Tsplit': 0.9658,
 'Tbot': 0.2603,
 'Tbot_a': 0.5368,
 'iber_a': 0.4244,
 'iber_pr_a': 0.1247,
 'iber_pr': 0.5396,
 'eura_a': 0.0248,
 'eura_pr': 0.1814,
 'theta': 19462667.95,
 'best_model': 'model_2_d'}

Try the ms function to build the ms command.
Reimporting the MS functions each time so any modifications to the my_ms_functions.py script are updated.

**no migration :**

In [215]:
if "my_ms_functions" in sys.modules:
    del sys.modules["my_ms_functions"]
import my_ms_functions

if par_dict["best_model"] == "model_1_a":
    print(my_ms_functions.model_1_a_ms(list(par_dict.values())[0:-1]))
if par_dict["best_model"] == "model_2_d":
    print(my_ms_functions.model_2_d_ms(list(par_dict.values())[0:-1]))


-t 99.379710 -I 2 1 1 -n 1 0.010419 -n 2 0.002668 -eg 0 1 -11.775132 -eg 0 2 -4.789848 -en 0.087196 1 0.029090 -eg 0.087196 1 -5.343199 -en 0.512404 1 0.282128 -en 0.512404 2 0.020450 -ej 1.082574 2 1 -r 3147.024166 993.797105


**migration from 2 to 1 :**

In [220]:
if "my_ms_functions" in sys.modules:
    del sys.modules["my_ms_functions"]
import my_ms_functions

if par_dict["best_model"] == "model_1_a":
    print(my_ms_functions.model_1_a_ms_m21(list(par_dict.values())[0:-1]))
if par_dict["best_model"] == "model_2_d":
    print(my_ms_functions.model_2_d_ms_m21(list(par_dict.values())[0:-1]))

-t 104.696670 -I 2 1 1 -n 1 0.035278 -n 2 0.002434 -eg 0 1 -6.447756 -eg 0 2 -5.667304 -ev 0.084201 2 1 0.455596 -en 0.111280 1 0.072297 -eg 0.111280 1 -4.765419 -en 0.478955 1 0.416924 -en 0.478955 2 0.019558 -ej 0.948179 2 1 -r 3315.394535 1046.966695


**migration from 1 to 2 :**

In [216]:
if "my_ms_functions" in sys.modules:
    del sys.modules["my_ms_functions"]
import my_ms_functions

if par_dict["best_model"] == "model_1_a":
    print(my_ms_functions.model_1_a_ms_m12(list(par_dict.values())[0:-1]))
if par_dict["best_model"] == "model_2_d":
    print(my_ms_functions.model_2_d_ms_m12(list(par_dict.values())[0:-1]))

-t 80.129225 -I 2 1 1 -n 1 0.046323 -n 2 0.003430 -eg 0 1 -4.514658 -eg 0 2 -4.637205 -ev 0.043854 1 2 0.332275 -en 0.153800 1 0.092759 -eg 0.153800 1 -3.526801 -en 0.659880 1 0.552717 -en 0.659880 2 0.035855 -ej 1.306687 2 1 -r 2537.425466 801.292252


this is how dadi does it from the *core* function :

In [12]:
import my_mscore_functions
mscore = my_mscore_functions.model_1_a_mscore(list(par_dict.values())[0:6])

In [16]:
nsamples = [1,1] # number of alleles to sample from each population of the simulation
reps = 1 # number of repetitions of the simulation
recomb = 0.2 # assumed recombination rate - for the simulated segment length
rsites= par_dict["theta"]*10 # sites for recombination - dadi defaults to 10*theta
mscommand = dadi.Misc.ms_command(par_dict["theta"], nsamples, mscore, reps, recomb=recomb, rsites=rsites)
mscommand

'ms 2 1 -t 1588900.750000 -I 2 1 1 -n 1 0.061631 -n 2 0.096090 -eg 0 2 -17.014774 -en 0.062837 1 0.840800 -en 0.062837 2 0.279900 -ej 0.858237 2 1 -r 0.200000 15889007'

In [15]:
par_dict["theta"]

1588900.75